# Getting Started with Pandas and DuckDB

This notebook will walk you through how to work with real NBA data using two powerful tools in Python:

- **DuckDB**, a super-fast in-process SQL database (like SQLite but for analytics)
- **Pandas**, a library for working with tables (called "DataFrames")

We’ll start by connecting to the database and then gradually learn how to filter, group, and analyse data using Python code.

Even if this is your first time seeing Python, don’t worry — each section includes an explanation of what’s happening and why.


In [1]:
# First, import the libraries we need
import duckdb  # lets us run SQL in Python
import pandas as pd  # used to work with tabular data

# Now, connect to our NBA database
con = duckdb.connect('../duckdb/nba.duckdb', read_only=True)

# This database is just a single file on your computer — no server needed


In [ ]:
# Let's see what tables exist in our database
# These are like sheets in Excel or tabs in Google Sheets
con.execute("SHOW TABLES").fetchdf()


Each row in the table above represents a dataset we can work with. We'll mostly use:

- `players_cleaned`: info about players (names, positions, height, age)
- `player_stats_cleaned`: performance data (points per game, assists, etc.)
- `top_scorers_by_season`: players with the highest average points (already filtered for us)


In [ ]:
# Load the full player stats table into a DataFrame (think: spreadsheet in code)
df_stats = con.execute("SELECT * FROM player_stats_cleaned").fetchdf()

# Let's look at the first 5 rows
df_stats.head()


This gives us a preview of the table as a Pandas DataFrame. Each row is one player. Each column is one stat (like `points_per_game` or `games_played`).

You can think of `df_stats` as a named spreadsheet inside your code.


In [ ]:
# .info() shows column names, data types, and whether there are missing values
df_stats.info()

In [ ]:
# .describe() gives basic statistics like min, max, mean, and quartiles
df_stats.describe()


These are your quick stats for all numeric columns:
- Count = how many values (not missing)
- Mean = average
- Std = how spread out the data is (standard deviation)
- Min/Max = lowest and highest values

Great for spotting weird values or getting a feel for the data.


In [ ]:
# Let's find players who played more than 50 games
df_stats[df_stats["games_played"] > 50]


This is called **boolean indexing**. We're telling pandas:
> Show me only the rows where `games_played` is greater than 50.

This is one of the most common and powerful tools in pandas: filtering your data to focus on just what you need.


In [ ]:
# Show the top 10 scorers by points per game
df_stats.sort_values("points_per_game", ascending=False).head(10)


We use `.sort_values()` to order the DataFrame.
- `"points_per_game"` is the column we're sorting by
- `ascending=False` means highest scores first
- `.head(10)` shows only the top 10 rows


In [ ]:
# Load player info so we can group by position
df_players = con.execute("SELECT * FROM players_cleaned").fetchdf()

# Merge player stats and player info using player_id (the shared key)
df_joined = pd.merge(df_stats, df_players, on="player_id")

# Now group by position and calculate average points per game
df_joined.groupby("position")["points_per_game"].mean().sort_values(ascending=False)


This is our first grouped analysis:
- `pd.merge(...)` combines the stats and player info into one table
- `.groupby("position")` splits the data into groups (one for each position)
- `["points_per_game"].mean()` calculates the average within each group

This is great for comparing categories.


### Visualisation Using Python

[matplotlib documentation](https://matplotlib.org/stable/index.html)

In this tutorial, we’ll use **matplotlib**, which is the most widely-used plotting library in Python.

matplotlib lets you create charts and plots directly from your data — just like you'd do in Excel or Google Sheets, but with code. You can use it to draw:
- Bar charts
- Line graphs
- Histograms
- Scatter plots
- And more

It’s especially useful when you're exploring data and want to **see patterns** or **communicate insights visually**.

---

In the example below, we use matplotlib (through a built-in pandas method) to create a **histogram** — a type of bar chart that shows how data is distributed across ranges.

This lets you quickly answer questions like:
- How common are 20+ PPG (points per game) players?
- Is the data skewed toward low or high scores?
- Are there any surprising gaps or outliers in the data?

You’ll see matplotlib pop up again and again in real-world data analysis — it’s a foundational skill worth getting comfortable with.


In [ ]:
import matplotlib.pyplot as plt

# Plot histogram of points per game
df_stats["points_per_game"].plot(kind="hist", bins=30, title="Points Per Game Distribution")

plt.xlabel("Points Per Game")
plt.ylabel("Number of Players")
plt.show()


## What You’ve Learned

- How to connect to a DuckDB database in Python
- How to read SQL tables into pandas DataFrames
- How to explore, filter, and sort tabular data
- How to group by categories and calculate averages
- How to visualise a distribution with a histogram

This is a great foundation for doing real-world data analysis.

Next steps:
- Try filtering by other columns (assists, rebounds)
- Build your own leaderboard
- Add visualisations like bar plots or scatter plots


# Homework: Explore the NBA Data

---

Here are some practice tasks to help you get more comfortable working with real data. These are designed to help you explore the dataset using the skills you've just learned.

Don’t worry about getting everything perfect — just try things out and see what you can discover.


## Task 1: Sorting by Other Stats

---

### What You’re Learning:
How to reorder the rows in your dataset based on the values in a specific column.

Sorting helps you identify top or bottom performers — for example, the players with the most minutes, highest number of rebounds, or fewest turnovers.


In [ ]:
# Example: Sort by most points
df_stats.sort_values("points_per_game", ascending=False).head(10)


### Task 1a: Answer the following quesitons
- Who are the top rebounders?
- Which players had the most steals?
- Who turned the ball over the most?

In [19]:
# Which player had the most rebounds?

In [20]:
# Which player had the most steals?

In [21]:
# Which player had the most turnovers?

- Most reboudns:
- Most steals: 
- Most turnovers:

## Task 2: Filtering Rows Based on a Condition

---

### What You’re Learning:
How to select only the rows that meet a specific condition — for example, players who scored more than a certain number of points, or played in a minimum number of games.

This is one of the most common operations in data analysis. It helps you focus on just the part of the dataset that matters for your question.


In [ ]:
# Example: Players with more than 1500 total points
df_stats[df_stats["points_per_game"] > 1500]

### Task 2a: Find all players who averaged more than 500 this season.

In [26]:
# Players with more than 500 assits

## Task 3: Filtering with Multiple Conditions
---

### What You’re Learning:
How to filter using more than one condition (AND / OR).

In [ ]:
# Example: Players with >1500 points AND >70 games played
df_stats[
    (df_stats["points_per_game"] > 1500) & 
    (df_stats["games_played"] > 70)
]

### Task 3a: Find players who played more than 60 games and had fewer than 200 total turnovers

In [30]:
# Players with more than 60 games played and less than 200 turnovers

## Task 4: Selecting Columns
---

### What You’re Learning:
How to view just the columns you care about.

In [ ]:

# Example: Show player name and total points
df_stats[["player_name", "points_per_game"]]

### Task 4a: Show a table with just `player_name`,`team _id`, `minutes_per_game`

In [34]:
# Columns: player_name, team_id, minutes_per_game

## Task 5: Creating New Columns
---


### What You’re Learning:
How to calculate your own stat using columns.

In [ ]:
# Example: Custom stat (PRA = total_points + total_assists + total_rebounds)
df_stats["score"] = (
    df_stats["points_per_game"] +
    df_stats["assists_per_game"] +
    df_stats["rebounds_per_game"]
)   

df_stats[["player_name", "score"]].sort_values(by="score", ascending=False)

### Task 5a: Create new columns for season averages


Right now, all of the stats in your DataFrame are **season totals** (e.g. total points, total assists).  
But sometimes it's more useful to compare players by how much they contributed **per game**.

In this task, you’ll create **new columns** that calculate per-game averages for each player.

---

#### What to Do:

- Create three new columns with per-game stats.
- Use clear names like `avg_points`, `avg_assists`, and `avg_rebounds`.

Once you’ve created the new columns, display a table showing each player’s name, number of games played, and their per-game averages.

---

#### Follow-Up

- Sort the DataFrame to find the players with the highest `avg_points`.
- What can you tell from the top of that list?

In [54]:
# Average points

In [55]:
# Average assists

In [56]:
# Average rebounds

### Task 6: Visualise the Top Scorers

Now that you’ve calculated average points per game, let’s visualise it.

- Note: use Google and the docs for matplotlib to figure out how to make a bar chart

---

#### Your Goal:

Create a **bar chart** that shows the top 10 players by `avg_points`.

---

#### What to Do:

- Sort the DataFrame by `avg_points`
- Select the top 10 players
- Plot a bar chart using:
  - Player name on the x-axis
  - Average points on the y-axis

Use your judgment to make the chart clear and readable. Add a title, adjust the labels if needed, and make sure the x-axis text isn’t cut off.

---

#### Follow-Up

- Try plotting a similar chart for `avg_assists` or `avg_rebounds`.
- What differences do you notice in who appears on each chart?


In [57]:
# Average points chart

In [58]:
# Average assists

In [59]:
# Average assists